In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/nfl.csv")
df.head()

,3d%,3d%_allowed,TOP,date,first_downs,first_downs_allowed,ha,margin,opp,pass_yards,...,rush_yards,rush_yards_allowed,sacked,sacks,takeaways,team,total_points,total_yards,total_yards_allowed,turnovers
0,0.307692,0.285714,28.350000,9/10/2009,18,19,away,-3,PIT,234,...,86,36,1,4,3,TEN,3,320,357,2
1,0.285714,0.307692,36.183333,9/10/2009,19,18,home,3,TEN,321,...,36,86,4,1,2,PIT,3,357,320,3
2,0.363636,0.400000,29.116667,9/13/2009,16,19,away,-12,ATL,163,...,96,68,4,2,0,MIA,26,259,281,4
3,0.400000,0.363636,30.883333,9/13/2009,19,16,home,12,MIA,213,...,68,96,2,4,4,ATL,26,281,259,0
4,0.250000,0.333333,26.550000,9/13/2009,10,16,away,5,CIN,227,...,75,86,3,3,2,DEN,19,302,307,0


In [3]:
# Dummify ha where away is 0 and home is 1
df["ha"] = df["ha"].map(lambda x: 0 if x == "away" else 1)

In [4]:
# Grab dummy variables
dummy_vars = pd.get_dummies(df[["opp", "team"]])
dummy_vars.head()

,opp_ARI,opp_ATL,opp_BAL,opp_BUF,opp_CAR,opp_CHI,opp_CIN,opp_CLE,opp_DAL,opp_DEN,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Add our dummy columns into the features df
df[list(dummy_vars.columns)] = dummy_vars

In [6]:
# define the features and label
X = df.drop(["result", "date", "opp", "team", "margin", "points", "points_allowed", "total_points"], axis=1)
y = df['result']
print(X.shape, y.shape)

(4806, 82) (4806,)


In [7]:
X.head()

,3d%,3d%_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0.307692,0.285714,28.350000,18,19,0,234,321,28,61,...,0,0,0,0,0,0,0,0,1,0
1,0.285714,0.307692,36.183333,19,18,1,321,234,60,70,...,0,0,0,0,1,0,0,0,0,0
2,0.363636,0.400000,29.116667,16,19,0,163,213,27,56,...,0,0,0,0,0,0,0,0,0,0
3,0.400000,0.363636,30.883333,19,16,1,213,163,35,65,...,0,0,0,0,0,0,0,0,0,0
4,0.250000,0.333333,26.550000,10,16,0,227,221,39,51,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X.columns

Index(['3d%', '3d%_allowed', 'TOP', 'first_downs', 'first_downs_allowed', 'ha',
       'pass_yards', 'pass_yards_allowed', 'penalty_yards', 'plays',
       'rush_yards', 'rush_yards_allowed', 'sacked', 'sacks', 'takeaways',
       'total_yards', 'total_yards_allowed', 'turnovers', 'opp_ARI', 'opp_ATL',
       'opp_BAL', 'opp_BUF', 'opp_CAR', 'opp_CHI', 'opp_CIN', 'opp_CLE',
       'opp_DAL', 'opp_DEN', 'opp_DET', 'opp_GBP', 'opp_HOU', 'opp_IND',
       'opp_JAX', 'opp_KCC', 'opp_LAC', 'opp_LAR', 'opp_MIA', 'opp_MIN',
       'opp_NEP', 'opp_NOS', 'opp_NYG', 'opp_NYJ', 'opp_OAK', 'opp_PHI',
       'opp_PIT', 'opp_SEA', 'opp_SFO', 'opp_TBB', 'opp_TEN', 'opp_WAS',
       'team_ARI', 'team_ATL', 'team_BAL', 'team_BUF', 'team_CAR', 'team_CHI',
       'team_CIN', 'team_CLE', 'team_DAL', 'team_DEN', 'team_DET', 'team_GBP',
       'team_HOU', 'team_IND', 'team_JAX', 'team_KCC', 'team_LAC', 'team_LAR',
       'team_MIA', 'team_MIN', 'team_NEP', 'team_NOS', 'team_NYG', 'team_NYJ',
       'team_OA

In [11]:
# only select one row
X = X.iloc[[0]]

In [12]:
X

,3d%,3d%_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0.307692,0.285714,28.35,18,19,0,234,321,28,61,...,0,0,0,0,0,0,0,0,1,0


In [13]:
oriented_df = X.to_dict(orient="records")

In [15]:
default_input_dict = oriented_df[0]

In [16]:
default_input_dict

{'3d%': 0.30769230800000003,
 '3d%_allowed': 0.28571428600000004,
 'TOP': 28.350000000000001,
 'first_downs': 18.0,
 'first_downs_allowed': 19.0,
 'ha': 0.0,
 'opp_ARI': 0.0,
 'opp_ATL': 0.0,
 'opp_BAL': 0.0,
 'opp_BUF': 0.0,
 'opp_CAR': 0.0,
 'opp_CHI': 0.0,
 'opp_CIN': 0.0,
 'opp_CLE': 0.0,
 'opp_DAL': 0.0,
 'opp_DEN': 0.0,
 'opp_DET': 0.0,
 'opp_GBP': 0.0,
 'opp_HOU': 0.0,
 'opp_IND': 0.0,
 'opp_JAX': 0.0,
 'opp_KCC': 0.0,
 'opp_LAC': 0.0,
 'opp_LAR': 0.0,
 'opp_MIA': 0.0,
 'opp_MIN': 0.0,
 'opp_NEP': 0.0,
 'opp_NOS': 0.0,
 'opp_NYG': 0.0,
 'opp_NYJ': 0.0,
 'opp_OAK': 0.0,
 'opp_PHI': 0.0,
 'opp_PIT': 1.0,
 'opp_SEA': 0.0,
 'opp_SFO': 0.0,
 'opp_TBB': 0.0,
 'opp_TEN': 0.0,
 'opp_WAS': 0.0,
 'pass_yards': 234.0,
 'pass_yards_allowed': 321.0,
 'penalty_yards': 28.0,
 'plays': 61.0,
 'rush_yards': 86.0,
 'rush_yards_allowed': 36.0,
 'sacked': 1.0,
 'sacks': 4.0,
 'takeaways': 3.0,
 'team_ARI': 0.0,
 'team_ATL': 0.0,
 'team_BAL': 0.0,
 'team_BUF': 0.0,
 'team_CAR': 0.0,
 'team_CHI': 0.0